In [ ]:
!pip install git+https://github.com/ultralytics/ultralytics.git@main
!pip install --upgrade opencv-python numpy pyexiftool pyproj flask flask-ngrok flask-cors pyngrok gdown
!ngrok update
!ngrok authtoken '' #put your token here
!apt-get install exiftool

  Cloning https://github.com/ultralytics/ultralytics.git (to revision main) to /tmp/pip-req-build-b7j2keff
  Running command git clone --filter=blob:none --quiet https://github.com/ultralytics/ultralytics.git /tmp/pip-req-build-b7j2keff
  Resolved https://github.com/ultralytics/ultralytics.git to commit e248e5b5062844ff358f8e754692cd1889efde56
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 107.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 64.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 52.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import logging
from flask import Flask, jsonify, request
import torch
import gc
import os
import uuid
import cv2
import numpy as np
import json
from pyproj import Transformer, CRS
import math
from ultralytics import YOLO
from pyngrok import ngrok
import gdown
import subprocess
import re

app = Flask(__name__)

# Configure logging
logging.basicConfig(level=logging.DEBUG, format='%(asctime)s - %(levelname)s - %(message)s')

# Camera Parameters for DJI FC6310
camera_params = {
    "focal_length": 8.8,  # in mm
    "sensor_size": (13.2, 8.8),  # in mm
    "image_size": (5472, 3648),  # in pixels
}

# Define model path
MODEL_PATH = "model_yolo11_50ep_v2.pt"

# Google Drive file ID (extract from the link)
GDRIVE_FILE_ID = "1BsR-fh5GASwgiyroj1bTj9JC51XEufLO"
MODEL_URL = f"https://drive.google.com/uc?id={GDRIVE_FILE_ID}"

# Check if model exists, if not, download it
if not os.path.exists(MODEL_PATH):
    logging.info("Downloading YOLO model from Google Drive...")
    try:
        gdown.download(MODEL_URL, MODEL_PATH, quiet=False)
        logging.info("YOLO model downloaded successfully.")
    except Exception as e:
        logging.error(f"Failed to download YOLO model: {e}")
        raise

# Load the model
try:
    model = YOLO(MODEL_PATH)
    logging.info("YOLO model loaded successfully")
except Exception as e:
    logging.error(f"Error loading YOLO model: {e}")
    raise

# Path to store uploaded images
UPLOAD_FOLDER = "images/"
os.makedirs(UPLOAD_FOLDER, exist_ok=True)

def process_image_with_yolo(image_path, model, patch_size=640, stride=640):
    large_image = cv2.imread(image_path)
    if large_image is None:
        logging.error(f"Failed to load {image_path}")
        return None

    # Convert BGR to RGB
    large_image = cv2.cvtColor(large_image, cv2.COLOR_BGR2RGB)

    # Convert image to a float32 PyTorch tensor
    image_tensor = torch.from_numpy(large_image).float().permute(2, 0, 1).unsqueeze(0) / 255.0

    # Run inference using YOLO 11
    results = model(image_tensor)

    # Extract detection results
    global_centers = []
    for result in results:
        for box in result.boxes:
            x_center, y_center, _, _ = box.xywh[0].tolist()
            global_centers.append([int(x_center), int(y_center)])

    logging.info(f"Detections for {image_path}: {len(global_centers)} objects found")
    return [os.path.basename(image_path), global_centers]

def get_metadata(file_path):
    filename = os.path.basename(file_path)
    all_metadata = {'filename': filename}

    try:
        # Run exiftool as a shell command
        result = subprocess.run(["exiftool", "-json", file_path], capture_output=True, text=True)

        if result.returncode != 0:
            raise Exception(f"ExifTool error: {result.stderr}")

        # Parse JSON output
        metadata_list = json.loads(result.stdout)
        if metadata_list:
            all_metadata.update(metadata_list[0])  # ExifTool returns a list

        return all_metadata

    except Exception as e:
        logging.error(f"Error retrieving metadata from {file_path}: {e}")
        return {'error': str(e)}

def convert_gps_to_decimal(gps_value):
    """Convert GPS coordinates from degrees, minutes, and seconds to decimal format."""
    try:
        if isinstance(gps_value, (int, float)):  # Already in correct format
            return float(gps_value)

        parts = str(gps_value).replace(" deg", "").replace("'", "").replace('"', "").split()
        degrees, minutes, seconds = map(float, parts[:3])
        decimal_value = degrees + (minutes / 60) + (seconds / 3600)

        # If 'S' or 'W' is in the original string, make it negative
        if "S" in gps_value or "W" in gps_value:
            decimal_value = -decimal_value

        return decimal_value
    except Exception as e:
        logging.error(f"Error converting GPS value {gps_value}: {e}")
        return None  # Return None if conversion fails

def extract_numeric_value(value):
    """Extracts numeric value from a string like '221.9 m Above Sea Level'."""
    try:
        match = re.search(r"[-+]?\d*\.\d+|\d+", str(value))  # Find numeric part
        if match:
            return float(match.group(0))  # Convert extracted number to float
        else:
            raise ValueError(f"Could not extract numeric value from '{value}'")
    except Exception as e:
        logging.error(f"Error extracting numeric value from '{value}': {e}")
        return None  # Return None if conversion fails

def calculate_gsd(sensor_size, image_size, altitude, focal_length):
    gsd_x = (sensor_size[0] * altitude) / (image_size[0] * focal_length)
    gsd_y = (sensor_size[1] * altitude) / (image_size[1] * focal_length)
    return gsd_x, gsd_y

def calculate_rotated_points(center, gcp_list, rotation_angle=0):
    rotation_angle_rad = np.deg2rad(rotation_angle)
    rotated_points = [{'type': 'center', 'longitude': center[0], 'latitude': center[1]}]
    for i, gcp in enumerate(gcp_list):
        rotated_gcp_x = (gcp[2] - center[0]) * np.cos(rotation_angle_rad) - (gcp[1] - center[1]) * np.sin(rotation_angle_rad) + center[0]
        rotated_gcp_y = (gcp[2] - center[0]) * np.sin(rotation_angle_rad) + (gcp[1] - center[1]) * np.cos(rotation_angle_rad) + center[1]
        rotated_points.append({'type': f'GCP {i+1}', 'longitude': rotated_gcp_x, 'latitude': rotated_gcp_y})
    return rotated_points

def scale_and_translate_polygon(points, average_ratio, image_width, image_height):
    new_center = [image_width / 2, image_height / 2]
    original_center = next(p for p in points if p['type'] == 'center')
    dx = new_center[0] - original_center['longitude']
    dy = new_center[1] - original_center['latitude']
    translated_points = []
    for point in points:
        if point['type'] == 'center':
            translated_points.append({'type': 'center', 'longitude': point['longitude'] + dx, 'latitude': point['latitude'] + dy})
        else:
            distance_x = point['longitude'] - original_center['longitude']
            distance_y = point['latitude'] - original_center['latitude']
            scaled_distance_x = distance_x * average_ratio
            scaled_distance_y = distance_y * average_ratio
            translated_gcp_x = original_center['longitude'] + scaled_distance_x + dx
            translated_gcp_y = image_height - (original_center['latitude'] + scaled_distance_y + dy)
            translated_points.append({'type': point['type'], 'longitude': translated_gcp_x, 'latitude': translated_gcp_y})
    return translated_points

def map_points_to_nearest_gcp(translated_points, gcps, gcp_list, img_width, img_height, max_distance=200):
    mapped_points = {}
    excluded_types = ['center']
    gcp_data = {int(item[0][3:]): {'latitude': item[1], 'longitude': item[2], 'altitude': item[3], 'name': item[0]} for item in gcp_list}
    for point in translated_points:
        if point['type'] in excluded_types:
            continue
        gcp_num = int(''.join(filter(str.isdigit, point['type'])))
        gcp_info = gcp_data.get(gcp_num)
        min_distance = float('inf')
        nearby_gcps = []
        for gcp in gcps:
            distance = np.linalg.norm(np.array([point['longitude'], point['latitude']]) - np.array(gcp))
            if distance <= max_distance:
                nearby_gcps.append(gcp)
                if distance < min_distance:
                    min_distance = distance
        if 0 <= point['longitude'] < img_width and 0 <= point['latitude'] < img_height:
            if nearby_gcps and gcp_info:
                closest_gcp = min(nearby_gcps, key=lambda g: np.linalg.norm(np.array([point['longitude'], point['latitude']]) - np.array(g)))
                mapped_points[gcp_info['name']] = {"x": int(closest_gcp[0]), "y": int(closest_gcp[1])}
            elif min_distance > max_distance and gcp_info:
                mapped_points[gcp_info['name']] = {"x": int(point['longitude']), "y": int(point['latitude'])}
    return mapped_points

@app.route("/", methods=["GET"])
def home():
    with torch.no_grad():
      torch.cuda.empty_cache()
    gc.collect()
    torch.cuda.ipc_collect()
    return "Flask server is running successfully via Ngrok!"

# Endpoint for uploading a user picture
# Dictionary to store original filenames mapped to UUIDs

filename_map = {}

@app.route('/upload', methods=['POST'])
def upload_image():
    if 'file' not in request.files:
        logging.error("No file part in the request")
        return jsonify({'error': 'No file part'}), 400

    file = request.files['file']

    if file.filename == '':
        logging.error("No selected file")
        return jsonify({'error': 'No selected file'}), 400

    if file:
        try:
            image_uuid = str(uuid.uuid4())
            original_filename = file.filename
            filename = f"{image_uuid}.jpg"
            file_path = os.path.join(UPLOAD_FOLDER, filename)
            file.save(file_path)
            filename_map[image_uuid] = original_filename
            logging.info(f"File {original_filename} uploaded successfully as {filename}")
            return jsonify({'message': 'File uploaded successfully', 'image_uuid': image_uuid}), 200
        except Exception as e:
            logging.error(f"Error saving file: {e}")
            return jsonify({'error': str(e)}), 500

@app.route('/predict/<image_uuid>', methods=['POST'])
def run_prediction(image_uuid):
    image_path = os.path.join(UPLOAD_FOLDER, f"{image_uuid}.jpg")

    if not os.path.exists(image_path):
        logging.error(f"Image with UUID {image_uuid} not found")
        return jsonify({'error': 'Image not found'}), 404

    # Expect JSON body with gcp_list and crs
    data = request.get_json()
    if not data or 'gcp_list' not in data:
        logging.error("GCP list not provided in request body")
        return jsonify({'error': 'GCP list required in JSON body'}), 400

    gcp_list = data['gcp_list']  # e.g., [["gcp01", 1854440.881, 627710.620, 43.215], ...]
    crs_epsg = data.get('crs', 'EPSG:32647')  # Default to Zone 47N if not provided

    try:
        # Process image with YOLO
        detection_result = process_image_with_yolo(image_path, model)
        if not detection_result:
            original_filename = filename_map.get(image_uuid, f"{image_uuid}.jpg")
            result = {"filename": original_filename, "gcps": {}}
            logging.info(f"No detections for {image_path}")
            return jsonify(result), 200

        filename, gcps_cv_list = detection_result
        original_filename = filename_map.get(image_uuid, filename)

        # Process metadata
        gps_metadata = get_metadata(image_path)
        img_width = gps_metadata['ImageWidth']
        img_height = gps_metadata['ImageHeight']
        flight_height = float(gps_metadata.get("RelativeAltitude", 120.0))
        gsd_x, gsd_y = calculate_gsd(camera_params["sensor_size"], camera_params["image_size"], flight_height, camera_params["focal_length"])
        average_ratio = 1 / gsd_x

        yaw = math.radians(float(gps_metadata.get("CameraYaw", 0)))
        north_orientation = float(gps_metadata.get("CameraYaw", 0))

        # Define CRS
        crs_geo = CRS("EPSG:4326").to_3d()  # Geographic coordinates from image metadata
        crs_utm = CRS(crs_epsg).to_3d()     # User-specified CRS (e.g., UTM zone)

        # Get image center coordinates and always convert to the specified CRS
        latitude = convert_gps_to_decimal(gps_metadata.get("GPSLatitude"))
        longitude = convert_gps_to_decimal(gps_metadata.get("GPSLongitude"))
        altitude = extract_numeric_value(gps_metadata.get("GPSAltitude", 120.0))  # Default to 120m if missing

        transformer = Transformer.from_crs(crs_geo, crs_utm, always_xy=True)
        utm_x, utm_y, utm_z = transformer.transform(longitude, latitude, altitude)
        center = (utm_x, utm_y)

        # Process GCPs (assumed to be in the same CRS as crs_epsg, e.g., UTM)
        rotated_points = calculate_rotated_points(center, gcp_list, north_orientation)
        translated_points = scale_and_translate_polygon(rotated_points, average_ratio, img_width, img_height)

        mapped_points = map_points_to_nearest_gcp(translated_points, gcps_cv_list, gcp_list, img_width, img_height)

        result = {
            "filename": original_filename,
            "gcps": mapped_points
        }

        logging.info(f"Prediction successful for image {image_path} with CRS {crs_epsg}")
        return jsonify(result), 200
    except Exception as e:
        logging.error(f"Error processing prediction: {e}")
        return jsonify({'error': str(e)}), 500

@app.route("/clear", methods=["DELETE"])
def clear_images():
    try:
        # Get list of files in UPLOAD_FOLDER
        files = os.listdir(UPLOAD_FOLDER)
        for file in files:
            file_path = os.path.join(UPLOAD_FOLDER, file)
            if os.path.isfile(file_path) and file.endswith('.jpg'):
                os.remove(file_path)
                logging.info(f"Deleted file: {file}")

        # Clear the filename_map
        filename_map.clear()
        logging.info("All images cleared and filename_map reset")
        return jsonify({"message": "All uploaded images cleared successfully"}), 200
    except Exception as e:
        logging.error(f"Error clearing images: {e}")
        return jsonify({"error": str(e)}), 500

if __name__ == '__main__':
    # Start ngrok tunnel
    public_url = ngrok.connect(5000, "http")
    print(f" * Ngrok tunnel available at: {public_url}")

    # Run Flask app
    app.run(port=5000, threaded=True)

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


Downloading...
From: https://drive.google.com/uc?id=1BsR-fh5GASwgiyroj1bTj9JC51XEufLO
To: /content/model_yolo11_50ep_v2.pt
100%|██████████| 5.33M/5.33M [00:00<00:00, 80.8MB/s]


 * Ngrok tunnel available at: NgrokTunnel: "https://656f-34-143-191-215.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [22/Mar/2025 14:30:15] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [22/Mar/2025 14:30:16] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [22/Mar/2025 14:36:13] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [22/Mar/2025 14:37:07] "POST /upload HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [22/Mar/2025 14:37:08] "POST /upload HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [22/Mar/2025 14:37:16] "POST /upload HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [22/Mar/2025 14:37:16] "POST /upload HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [22/Mar/2025 14:37:24] "POST /upload HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [22/Mar/2025 14:37:24] "POST /upload HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [22/Mar/2025 14:37:32] "POST /upload HTTP/1.1" 20


Ultralytics 8.3.94 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (NVIDIA L4, 22693MiB)

0: 3648x5472 (no detections), 197.7ms
Speed: 51.1ms preprocess, 197.7ms inference, 256.1ms postprocess per image at shape (1, 3, 3648, 5472)


...

INFO:werkzeug:127.0.0.1 - - [22/Mar/2025 14:37:57] "POST /predict/9c403af1-c4e6-4baa-99ae-8b045c76981b HTTP/1.1" 200 -




0: 3648x5472 2 gcps, 184.3ms
Speed: 51.2ms preprocess, 184.3ms inference, 107.4ms postprocess per image at shape (1, 3, 3648, 5472)


INFO:werkzeug:127.0.0.1 - - [22/Mar/2025 14:37:58] "POST /predict/c43c173a-9eaf-428a-b1a4-28210a97019a HTTP/1.1" 200 -


0: 3648x5472 2 gcps, 185.1ms
Speed: 51.0ms preprocess, 185.1ms inference, 107.5ms postprocess per image at shape (1, 3, 3648, 5472)


INFO:werkzeug:127.0.0.1 - - [22/Mar/2025 14:37:59] "POST /predict/54b27100-c32e-44e6-8ef7-5e5425029602 HTTP/1.1" 200 -
